<center><h1 style="font-family: 'Georgia'; color: #f2f2f2; background-color:#800040; padding: 20px;">RFM Analysis: Online Store Customer Descriptive Segmentation</h1></center>

<p style="font-family: 'Georgia'; font-size: 16px; font-weight: 800; color: #800040;">
Customer Segmentation by KMeans Clustering.
</p>

<p style="font-family: 'Georgia'; font-size: 14px; font-weight: 500; color: #800040;">
dataset link: https://www.kaggle.com/datasets/yasserh/customer-segmentation-dataset
</p>

<p style="font-family: 'Georgia'; font-size: 14px; font-weight: 500; color: #800040;">
RFM (Recency, Frequency, Monetary) analysis is vital for businesses as it helps identify their most valuable customers by evaluating their purchasing behavior. By segmenting customers based on these criteria, businesses can tailor personalized marketing strategies and optimize customer retention, ultimately enhancing overall profitability.
</p>



<h1 style="font-family: 'Georgia'; font-size: 24px; color: #008000;">Preprocessing</h1>

In [1]:
# IMPORT LIBRARIES
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
# LOAD DATASET
df = pd.read_excel('../1_datasets/original_datasets/customer_segmentation.xlsx')
df.head(10)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
5,536365,22752,SET 7 BABUSHKA NESTING BOXES,2,2010-12-01 08:26:00,7.65,17850.0,United Kingdom
6,536365,21730,GLASS STAR FROSTED T-LIGHT HOLDER,6,2010-12-01 08:26:00,4.25,17850.0,United Kingdom
7,536366,22633,HAND WARMER UNION JACK,6,2010-12-01 08:28:00,1.85,17850.0,United Kingdom
8,536366,22632,HAND WARMER RED POLKA DOT,6,2010-12-01 08:28:00,1.85,17850.0,United Kingdom
9,536367,84879,ASSORTED COLOUR BIRD ORNAMENT,32,2010-12-01 08:34:00,1.69,13047.0,United Kingdom


In [3]:
# CHECK NULL VALUES
df.isnull().sum()

InvoiceNo           0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     135080
Country             0
dtype: int64

<p style="font-family: 'Georgia'; font-size: 14px; color: #800040;">
Since this analysis is primarily focused on customer behavior based on total spending, frequency of purchase, and recency of purchase, and the CustomerID column is the key identifier in this analysis, then it would be advisable to drop rows with missing values in the CustomerID column. This will ensure that we have complete and accurate data for this analysis, which is crucial for generating reliable insights.
</p>

In [4]:
# DROP NULL VALUES
df.dropna(inplace=True)
print(df.shape)

(406829, 8)


In [5]:
# CHECK FOR DUPLICATE RECORDS
df.duplicated().sum()

5225

In [6]:
# DROP DUPLICATED VALUES
df.drop_duplicates(inplace=True)
print(df.shape)

(401604, 8)


In [7]:
# DROP IRRELEVANT COLUMNS
df.drop(['Description', 'Country'], axis=1, inplace=True)

In [8]:
print(df.columns)

Index(['InvoiceNo', 'StockCode', 'Quantity', 'InvoiceDate', 'UnitPrice',
       'CustomerID'],
      dtype='object')


<p style="font-family: 'Georgia'; font-size: 14px; color: #800040;">
    We decided to drop the Description column because it is the same as StockCode. In store settings, they have SKUs or Stock Keeping Units which allows you to keep track of each individual stock as opposed to have lengthy descriptive words. In this way, tracking is easier and making analyses like theses becomes simpler.
</p>
<p style="font-family: 'Georgia'; font-size: 14px; color: #800040;">
On the other hand, we dropped the country column because we want this analysis to be applicable globally. This only makes perfect sense because online stores are usually not constrained to just domestic customers.
</p>

In [9]:
# EXAMINE THE DATASET SHAPE
print('Data Shape:', df.shape)

Data Shape: (401604, 6)


In [10]:
# EXAMINE DATATYPES
df.dtypes

InvoiceNo              object
StockCode              object
Quantity                int64
InvoiceDate    datetime64[ns]
UnitPrice             float64
CustomerID            float64
dtype: object

<p style="font-family: 'Georgia'; font-size: 14px; color: #800040;">
CustomerId column should be object data type, so we need to change it. It is laso a good idea to remove the .0 at the end of each customer id, so lets do that as well.
</p>

In [11]:
# CHANGE DATATYPES
import warnings
warnings.filterwarnings('ignore')

df['CustomerID'] = df['CustomerID'].astype('str')
df['CustomerID'] = df['CustomerID'].str.replace('.0', '')
df['CustomerID'] = df['CustomerID'].astype('object')

In [12]:
df.dtypes

InvoiceNo              object
StockCode              object
Quantity                int64
InvoiceDate    datetime64[ns]
UnitPrice             float64
CustomerID             object
dtype: object

<h1 style="font-family: 'Georgia'; font-size: 24px; color: #008000;">Creating a New Data Frame</h1>

<p style="font-family: 'Georgia'; font-size: 14px; color: #800040;">
My approach in this analysis is to create a new dataframe, RFM, with the following columns:
</p>

<p style="font-family: 'Georgia'; font-size: 14px; color: #800040; margin-left: 18px;">
1. CustomerID, which are all the unique customers we have in df.
</p>

<p style="font-family: 'Georgia'; font-size: 14px; color: #800040; margin-left: 18px;">
2. Recency of purchase, which we will get from the InvoiceDate mapped for each customer. 
</p>

<p style="font-family: 'Georgia'; font-size: 14px; color: #800040; margin-left: 18px;">
3. Frequecy of purchase, which we will get from counting the Invoice records for each individual customer.
</p>

<p style="font-family: 'Georgia'; font-size: 14px; color: #800040; margin-left: 18px;">
4. Total Amount Purchased, which will be obtained by summing the total amounts spent by the customer on all the products they have purchased. Each component in this sum is calculated as the product of the quantity purchased and the unit price of each item.
</p>

<p style="font-family: 'Georgia'; font-size: 14px; color: #800040;">
This is a possible approach because in the data we have, each customer is tracked individually. They have a known identifier. In this way, we can tell which group of customers behave the same way.
</p>

In [13]:
# USE A PANDAS SERIES TO STORE ALL OF UNIQUE CUSTOMERS
customers_series = df['CustomerID'].unique()

<p style="font-family: 'Georgia'; font-size: 14px; color: #800040;">
I thought it will be a great idea to have a list of all unique customers right away so that we just need to reference them later and not find them at the same time of extracting relevant information about them.
</p>

In [14]:
# THE FUNCTION THAT WILL GENERATE THE df_new 
def generate_data_frame(customers_series, df):
    extracted_data = []
    base_date = '2012-01-01'
    base_date = pd.to_datetime(base_date) #we need it to be in datetime format so we can extract the days
    
    for customer_id in customers_series:
        specific_customer = df[df['CustomerID'] == customer_id].copy() # check all records for a specific customer
        
        #recency
        most_recent = specific_customer['InvoiceDate'].max() # last transaction made
        recency = (base_date - most_recent).days # this works because we are working with datatime formats
    
        #frequency
        frequency = specific_customer['InvoiceNo'].nunique() # count the number of transactions a specific customer has
        
        #monetary
        specific_customer.loc[:, 'TotalPrice'] = specific_customer['Quantity'] * specific_customer['UnitPrice']  # calculate the total spending for each row
        total_spending = specific_customer['TotalPrice'].sum()  # sum the total spending for the specific customer
        total_spending = "{:.2f}".format(total_spending)

        extracted_data.append((customer_id, recency, frequency, total_spending))
        
    cleaned_df = pd.DataFrame(columns=['CustomerID', 'Recency', 'Frequency', 'Monetary'], data=extracted_data)
    
    return cleaned_df

In [16]:
RFM = generate_data_frame(customers_series, df)
RFM.head(10)

,CustomerID,Recency,Frequency,Monetary
0,178,23,156,33817.17
1,147,23,115,36128.77
2,12583,24,18,7187.34
3,13748,117,5,948.25
4,150,30,23,4628.00
5,15291,47,20,4596.51
6,14688,29,27,5055.61
7,179,26,111,30533.27
8,15311,22,118,59284.19
9,14527,24,86,7709.69


<h1 style="font-family: 'Georgia'; font-size: 24px; color: #008000;">Save Dataset</h1>

In [18]:
# SAVE CLEANED DATASET TO CSV
RFM.to_csv('../1_datasets/cleaned_datasets/RFM.csv', index=False)